# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from pandas.api.types import CategoricalDtype
from implicit.nearest_neighbours import bm25_weight, tfidf_weight
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [5]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 90386 to 4938


In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818980,818981,819063,819255,819304,819308,819330,819518,819594,819765,...,15926927,15927403,15927850,15972074,15972298,16100266,16729415,16770156,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

-----

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [14]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        self.data = data
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = \
            self.prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data):
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )
        user_item_matrix = user_item_matrix.astype(float)

        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(user_item_matrix.T)

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(user_item_matrix.T)

        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        res = [self.id_to_itemid[rec[0]] for rec in
               self.model.recommend(userid=self.userid_to_id[user],
                                    user_items=self.user_item_matrix,  # на вход user-item matrix
                                    N=N,
                                    filter_already_liked_items=False,
                                    filter_items=[999999],  # !!!
                                    recalculate_user=True)]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        similar_users = [self.id_to_userid[i] for i, _ in self.model.similar_users(self.userid_to_id[user], N=6)[1:]]

        result = self.data.query('(user_id in @similar_users) & (item_id != 999999)').groupby(['user_id', 'item_id'])[
            'quantity'].count().reset_index()

        result.sort_values('quantity', ascending=False, inplace=True)

        res = list(result['item_id'].values[:N])

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


----

Проверка, что все работает

In [15]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
# from src.recommenders import MainRecommender

recommender = MainRecommender(prefilter_items(data, item_features))

In [16]:
recommender.get_similar_users_recommendation(2)

[1106523, 962229, 1053690, 1125123, 8090521]

In [17]:
recommender.get_similar_items_recommendation(3)

[844165, 1053690, 1092026, 8090537, 962229]

### Дополнительное домашнее задание*
**Важно:** для получения оценки "Хорошо" достаточно выполнить все указанное выше, чтобы получить "Отлично" необходимо выполнить хотя бы половину этого задания (описать пайплайн планируемых действий и выполнить преобразование датасета в понятный для рекомендательной системы вид, в соответствии с требованиями, см. ниже).

Это часть дз максимально приближена к реальным задачам из области рекомендательных систем. Цель этой части научить вас строить полноценный пайплайн, от предобработки данных, до получения MVP рекомендательной системы (датасет + обученная на нем модель).

В качестве исходных данных вам дан датасет из ритейла (отличный от того который мы рассматриваем на курсе), его можно найти по пути (data/events.csv.zip).

Задача звучит так:
"Компания, занимающаяся ритейлом, хочет запустить рассылку с индивидуальными рекомендациями, для теста вам как специалисту в области DS готовы предоставить данные о событиях совершенных пользователями, единственной целью на данном этапе является увеличение выручки".

*Отступление:* к сожалению в реальной жизни DS это не про модели и нейронные сети, в большинстве случаев вам придется работать с данными, очень много и данные могут быть различного качества (в большинстве случаев не самого хорошего 🗑). Поэтому очень важную часть в работе DS занимает проектирование и создание датасета. Обычно это происходит по следующему пайплайну:
   1. Выгрузка данных
   2. Разведочный анализ (поиск аномалий, оценка аудитории и ее сегмантация, поиск инсайтов)
   3. Проектирование датасета (для проектирование датасета необходимо понимание моделей которые используются в этой области, помимо этого нужно опредеиться что будет использоваться в качестве таргета и какие признаки стоит добавлять, а каие не имеет смысла)
   4. Создание бэйзлайна (важная часть которую легко упустить, но без нее невозможно доказать состоятельность модели)
   5. Создание MVP
   1. Продолжаем улучшение в попытке побить бэйзлайн или линейную модель :)
   
Обязательные требования к заданию:
1. В данном задании разведочный анализ является необязательным, его можно пропустить.
1. Необходимо описать последовательность ваших действий (3-5 пунктов, иными словами сформировать задачу с технической стороны, как бы вы преподнесли ее бизнесу)
1. Создание user_item_matrix должно происходить без использования сводных таблиц (pd.pivot), совет: coo_matrix или просто гугл.
2. Создать бэйзлайн
1. Использовать любую из моделей которую мы с вами изучили ранее

*Зачем вам это?*

Если вы реализуете все вышеуказанное и получете средний для рекомендательных систем результат > 10%, то это будет отличным пет-проектом который будет не стыдно указать при устройстве на работу, озаглавив его как "Создание рекомендательной системы для ритейла с 0".

Крутые фичи которые можно добавить** (это прям супер дополнительно и сможет пригодиться только вам):
1. Добавить метод для рекомендаций похожих товаров (подумать как можно оценить данную систему)
1. Провести разведочный анализ (все любят картинки и графики)
1. Добавить что-нибудь от себя!

In [125]:
df = pd.read_csv('data/events.csv.zip', compression='zip')

In [126]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [127]:
df.event.unique()

array(['view', 'addtocart', 'transaction'], dtype=object)

In [128]:
event_replacement = dict(zip(df.event.unique(), [1, 3, 6]))
event_replacement

{'view': 1, 'addtocart': 3, 'transaction': 6}

In [129]:
df['event'].replace(event_replacement, inplace=True)

In [134]:
print(f'Interactions: {df.shape[0]}',
      f'Num unique users: {df.visitorid.nunique()}',
      f'Num unique items: {df.itemid.nunique()}',
      sep='\n'
     )

Interactions: 2756101
Num unique users: 1407580
Num unique items: 235061


In [135]:
weighted_popularity = df.groupby('itemid', as_index=False)['event'].sum().sort_values('event', ascending=False)

### В наборе данных есть 3 уникальных взаимодействия: просмотр, добавление в корзину, покупка. Выставлю им веса {1, 3, 6}

### 235 тысяч уникальных товаров, оставлю 5 тысяч самых популярных с учетом веса взаимодействий

### Я создам 3 User-Item матрицы, элементами в которых будут суммы взаимодействий между пользователем и товаром. Две из них я взвешу с помощью tf-idf и bm25, одну оставлю как есть. Разложу эти 3 матрицы с помощью ALS, получу три модели, протестирую точность их предсказаний 

In [137]:
def reduce_items(data, n_keep=5000, weighted_popularity=weighted_popularity):
    top = weighted_popularity.head(n_keep)['itemid'].values
    data.loc[~data['itemid'].isin(top), 'itemid'] = 999999
    return data

In [138]:
def get_user_item_matrix(data=df, n_keep=5000, weights=None):
    data = reduce_items(data, n_keep)
    users_uniq = CategoricalDtype(sorted(data.visitorid.unique()), ordered=True)
    items_uniq = CategoricalDtype(sorted(data.itemid.unique()), ordered=True)
    row = data.visitorid.astype(users_uniq).cat.codes
    col = data.itemid.astype(items_uniq).cat.codes
    matrix = csr_matrix((data['event'], (row, col)), 
                              shape=(users_uniq.categories.size, items_uniq.categories.size))
    if weights == 'bm25':
        return bm25_weight(matrix.T).T
    elif weights == 'tfidf':
        return tfidf_weight(matrix.T).T
    return matrix

In [139]:
def get_fitted_model(user_item_matrix):
    model = AlternatingLeastSquares(factors=20, regularization=1e-3, iterations=15, num_threads=4)
    model.fit(user_item_matrix.T)
    return model

In [141]:
user_item_matrix = get_user_item_matrix(df)

In [142]:
user_item_matrix_tfidf = get_user_item_matrix(df, weights='tfidf')
user_item_matrix_bm25 = get_user_item_matrix(df, weights='bm25')

In [143]:
model_als = get_fitted_model(user_item_matrix)

In [144]:
model_tfidf = get_fitted_model(user_item_matrix_tfidf)
model_bm25 = get_fitted_model(user_item_matrix_bm25)

In [156]:
users_uniq = CategoricalDtype(sorted(df.visitorid.unique()), ordered=True)
items_uniq = CategoricalDtype(sorted(df.itemid.unique()), ordered=True)

In [158]:
users_init = users_uniq.categories.tolist()
items_init = items_uniq.categories.tolist()

In [160]:
users_new = np.arange(len(users_init))
items_new = np.arange(len(items_init))

In [161]:
userid_to_id = dict(zip(users_init, users_new))
id_to_userid = {j: i for i, j in userid_to_id.items()}

itemid_to_id = dict(zip(items_init, items_new))
id_to_itemid = {j: i for i, j in itemid_to_id.items()}

In [162]:
def get_recommendations(u_id, model=model_als, matrix=user_item_matrix, num=5):
    recs = [id_to_itemid[rec[0]] for rec in 
                                    model.recommend(u_id, 
                                    user_items=matrix,
                                    N=num, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],
                                    recalculate_user=True)]
    return recs

In [163]:
_, df_test = train_test_split(df, test_size=0.1)

In [244]:
result = df_test.groupby('visitorid')['itemid'].unique().reset_index()

In [224]:
result = result[result['itemid'].astype(str) != '[999999]']

In [228]:
%%time
result['recs_als'] = result['visitorid'].apply(get_recommendations)

CPU times: user 1min 9s, sys: 10.3 s, total: 1min 19s
Wall time: 20.2 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [231]:
result.head()

,visitorid,itemid,recs_als
1,2,[325215],"[379520, 315545, 422425, 369447, 219512]"
2,6,[65273],"[65273, 316753, 339703, 409804, 253615]"
4,31,[465565],"[5411, 325310, 241555, 9877, 96924]"
9,50,[331168],"[369447, 303715, 219512, 96924, 187946]"
10,51,[49967],"[7943, 37029, 370653, 318333, 299222]"


In [234]:
%%time
result['recs_bm25'] = result['visitorid'].apply(lambda x: get_recommendations(x, model_bm25, user_item_matrix_bm25))

CPU times: user 52min 40s, sys: 7min 47s, total: 1h 27s
Wall time: 15min 13s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [235]:
%%time
result['recs_tfidf'] = result['visitorid'].apply(
    lambda x: get_recommendations(x, model_tfidf, user_item_matrix_tfidf))

CPU times: user 52min 34s, sys: 8min 4s, total: 1h 38s
Wall time: 15min 24s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [243]:
result.head()

,visitorid,itemid,recs_als,recs_bm25,recs_tfidf
1,2,[325215],"[379520, 315545, 422425, 369447, 219512]","[259884, 29940, 449391, 342264, 216305]","[216305, 231243, 379520, 138427, 259884]"
2,6,[65273],"[65273, 316753, 339703, 409804, 253615]","[344723, 325852, 236180, 234493, 78163]","[65273, 316753, 253615, 339703, 22436]"
4,31,[465565],"[5411, 325310, 241555, 9877, 96924]","[465565, 159380, 264604, 447661, 17419]","[7943, 461686, 298009, 416017, 89775]"
9,50,[331168],"[369447, 303715, 219512, 96924, 187946]","[168952, 303833, 277762, 286645, 302329]","[232509, 151444, 429094, 357054, 341482]"
10,51,[49967],"[7943, 37029, 370653, 318333, 299222]","[227880, 272455, 76617, 49967, 83366]","[434782, 445351, 272455, 17114, 320130]"


In [237]:
result.to_csv('result.csv')

In [239]:
result.apply(lambda i: precision_at_k(i['recs_als'], i['itemid']), axis=1).mean() * 100

2.7917778043184733

In [240]:
result.apply(lambda i: precision_at_k(i['recs_tfidf'], i['itemid']), axis=1).mean() * 100

3.9805193943489017

In [241]:
result.apply(lambda i: precision_at_k(i['recs_bm25'], i['itemid']), axis=1).mean() * 100

9.43853921997405

### BM25 взвешивание показывает наилучший результат, но предсказания происходят очень долго, в отличии от использования не взвешенной матрицы